## Notebook 1.2 - Invoke KServe Endpoint

In this notebook, we will invoke the KServe endpoint created by
the accompanied notebook `1.1_deploy_torchserve_kserve.ipynb`.

In [ ]:
import os
import json
import base64
from base64 import b64encode 
import requests 

from kserve import KServeClient
from kubeflow.training.utils import utils


In [ ]:
# the endpoint name must be the same as the one created 
# in 1.1_deploy_torchserve_kserve.ipynb (or EKS execution of 2_create_pipeline_k8s_sagemaker.ipynb)
inference_endpoint = "image-classify"
namespace = utils.get_default_target_namespace()

client = KServeClient()

resp = client.get(
    inference_endpoint, 
    namespace=namespace
)

CLUSTER_IP = resp['status']['url']
MODEL_NAME = os.environ.get("MODEL_NAME", "cifar")
isvc_url = f"{CLUSTER_IP}/v1/models/{MODEL_NAME}:predict"

print(isvc_url)


In [ ]:
ENCODING = 'utf-8'
IMAGE_NAME = '0002.png'

# Read the Image binary
# result: bytes
with open(IMAGE_NAME, 'rb') as open_file:
    byte_content = open_file.read()

# base64 encode read data
# result: bytes 
base64_bytes = b64encode(byte_content)

# Decode bytes to text
# result: string (in utf-8)
base64_string = base64_bytes.decode(ENCODING)

# Create json data
raw_data={'instances':[{'data': base64_string}]}


In [ ]:
response=requests.post(
    isvc_url, 
    json=raw_data
)

print("Sending request to:", isvc_url)

print("Status Code", response)
if response == 200:
    print("JSON Response ", json.dumps(response.json(), indent=2)) 
    
labels = [
    'airplane', 
    'automobile', 
    'bird', 
    'cat', 
    'deer', 
    'dog',
    'frog', 
    'horse', 
    'ship', 
    'truck'
]


print(response.json()['predictions'])
print(labels[response.json()['predictions'][0]])
